# 頻譜分析與濾波處理
本 Notebook 展示了DPL如何對信號進行濾波處理與頻譜分析，並可視化處理結果。

## 功能
1. 載入信號數據
2. 計算取樣率
3. 使用運行數位濾波器進一步處理
4. 快速傅立葉變換（FFT）進行頻譜分析
5. 可視化頻譜結果


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import sosfilt, butter, lfilter

# 定義運行平均濾波器的窗口長度
idx = 20
data = np.loadtxt(r"filename", delimiter=',')
offset = int(data.size*0.05)
signal = data[offset:-offset,1]
time = data[offset:-offset,0]


## 計算取樣率
由於ESP32本身CLK的缺陷，$f_s$相當的不穩定，故我們動態修正不同次實驗的$f_s$以最佳化實驗結果。
修正公式如下:
$$
f_s = \frac{N}{T}
$$

其中：
- $f_s$ 是取樣率（Sampling Rate）。
- $N$  是取樣點數。
- $T$  是信號總持續時間。(由`time()`函式計算)

In [ ]:
sampling_rate = data.shape[0]/(data[-1,0]-data[0,0])
print('Sampling rate:', sampling_rate)


## 運用數位濾波器


In [ ]:
# 定義濾波器參數
cutoff_frequency = 0.1  # in Hz
order = 8  # Higher filter order

# Design high-pass filter using Butterworth with higher order
sos = butter(order, cutoff_frequency, btype='high', fs=sampling_rate, output='sos')

# 使用之前設計的濾波器係數進行濾波
filtered_signal = sosfilt(sos, signal)


signal2 = np.abs(filtered_signal)

window_length = 100
b = np.ones(window_length) / window_length  # 分子係數
a = 1  # 分母係數

# 使用 lfilter 應用運行平均濾波器
signal3 = lfilter(b, a, signal2)

## 快速傅立葉變換（FFT）
對信號進行頻譜分析，並找到最高頻率分量。


In [ ]:
signal4 = np.abs(np.fft.fft(signal))

checkhz_len = int(5/(sampling_rate/signal4.size)) #最高檢查5H整
max_indx = (np.argmax(signal4[1:checkhz_len]) + 1)
max_hz = max_indx * sampling_rate/signal4.size
max_spec = signal4[max_indx]
print('Maximum Frequency:', max_hz)


## 頻譜可視化
繪製信號的頻譜圖，並標註最大頻率分量。


In [ ]:
plt.ylabel('Amplitude', fontsize=15)
plt.xlabel('Frequency [Hz]', fontsize=15)
plt.title('Frequency Spectrum', fontsize=20)
plt.plot(sampling_rate/signal4.size*np.arange(1, checkhz_len), signal4[1:checkhz_len])
plt.plot(max_hz, max_spec, 'r*')
plt.show()
